In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,2185
2,Huelva,Confirmados PDIA 14 días,615
3,Huelva,Tasa PDIA 14 días,"120,41281035667645"
4,Huelva,Confirmados PDIA 7 días,302
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,2
629,Municipio de Huelva sin especificar,Total Confirmados,32
630,Municipio de Huelva sin especificar,Curados,4


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  2185.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  583.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1330 personas en los últimos 7 días 

Un positivo PCR cada 704 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,2185.0,302.0,615.0,510743.0,59.129543,120.412810,90.0
Huelva-Costa,1142.0,190.0,346.0,288115.0,65.945890,120.090936,57.0
Condado-Campiña,729.0,71.0,191.0,155057.0,45.789613,123.180508,24.0
Huelva (capital),583.0,108.0,204.0,143663.0,75.175933,141.998984,21.0
Lepe,169.0,31.0,53.0,27431.0,113.010827,193.212059,12.0
Sierra de Huelva-Andévalo Central,282.0,39.0,69.0,67571.0,57.717068,102.114813,11.0
Almonte,76.0,12.0,21.0,24191.0,49.605225,86.809144,7.0
Cartaya,41.0,10.0,15.0,19974.0,50.065085,75.097627,6.0
Cerro de Andévalo (El),14.0,9.0,9.0,2364.0,380.710660,380.710660,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),17.0,14.0,15.0,516.0,2713.178295,2906.976744,4.0
Villarrasa,29.0,6.0,19.0,2176.0,275.735294,873.161765,2.0
Villanueva de los Castillejos,20.0,2.0,13.0,2820.0,70.921986,460.992908,NaN
San Juan del Puerto,74.0,3.0,37.0,9300.0,32.258065,397.849462,0.0
Cerro de Andévalo (El),14.0,9.0,9.0,2364.0,380.710660,380.710660,6.0
Palma del Condado (La),108.0,17.0,33.0,10761.0,157.977883,306.662950,4.0
Cortegana,22.0,9.0,12.0,4666.0,192.884698,257.179597,0.0
Marines (Los),1.0,1.0,1.0,396.0,252.525253,252.525253,NaN
Zalamea la Real,28.0,2.0,7.0,3068.0,65.189048,228.161669,0.0
